In [117]:
import pandas as pd
import pathlib
import math
from collections import defaultdict

ALL_ZONES = "zonas_lat_long.csv"
ALL_VOTES = "resultado_presidente.csv"

zones = pd.read_csv(ALL_ZONES, encoding="latin1") #, index_col="numero_zona")
votes = pd.read_csv(ALL_VOTES, encoding="latin1", delimiter=";", \
                    usecols=["NR_TURNO", "SG_UF", "NR_ZONA", "NR_SECAO", \
                            "NM_VOTAVEL", "QT_VOTOS"])

# Ignore international votes
votes = votes[votes["SG_UF"] != "ZZ"]

In [118]:
zones.head()

,numero_zona,sigla_uf.1,lat_0,lng_0
0,60,MT,-13.662917,-57.888985
1,36,MT,-12.290314,-55.295626
2,20,MT,-15.647176,-56.130580
3,49,MT,-15.647176,-56.130580
4,26,MT,-14.667030,-52.352839


In [124]:
def save_merged_results(votes, zones, numero_turno="both", drop_na=True):
    if numero_turno != "both":
        votes = votes[votes["NR_TURNO"] == numero_turno]
    merged_data = votes.merge(zones, left_on=["SG_UF", "NR_ZONA"], \
                              right_on=["sigla_uf.1", "numero_zona"], how="right")
    idxs = merged_data["lat_0"].isna()
    if sum(idxs):
        print(f"There were {sum(idxs)} (={sum(idxs)/len(merged_data):.2f}%) "\
            "rows with invalid state initials/zone")
        if drop_na:
            merged_data = merged_data[~idxs]
    
    return merged_data

In [127]:
df = save_merged_results(votes, zones, numero_turno=1)
df.to_csv("Voting_lat_long.csv")

There were 1298 (=0.00%) rows with invalid state initials/zone
